In [0]:
import matplotlib.pyplot as plt
import numpy as np
import math
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import datasets
from tensorflow.keras import backend as K
 

# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.0004
    drop = 0.5
    epochs_drop = 5.0
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate



batch_size = 100
epochs = 50
input_dim = 100
sample_size = 6


(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

x_train = x_train.astype('float32') - 127.5
x_test = x_test.astype('float32') - 127.5
x_train /= 127.5
x_test /= 127.5
x_train = np.expand_dims(x_train, axis=-1)  # --> [6000,28,28,1]

In [0]:


# ************************************
#          discriminator
# ************************************
discriminator = models.Sequential()
discriminator.add(layers.Conv2D(64, (5, 5), padding='same', activation='tanh',
                                input_shape=(28, 28,1)))
discriminator.add(layers.MaxPooling2D(pool_size=(2, 2)))
discriminator.add(layers.Conv2D(128, (5, 5), padding='same', activation='tanh'))
discriminator.add(layers.MaxPooling2D(pool_size=(2, 2)))
discriminator.add(layers.Flatten())
discriminator.add(layers.Dense(1024, activation='tanh'))
discriminator.add(layers.Dense(1, activation='sigmoid'))

opt_D = optimizers.RMSprop(lr=0.0004, clipvalue=1.0)
discriminator.trainable = True
discriminator.compile(loss='binary_crossentropy', optimizer=opt_D)
discriminator.summary()



# ************************************
#          generator
# ************************************
generator = models.Sequential()
generator.add(layers.Dense(1024, activation='tanh', input_dim=input_dim))
generator.add(layers.Dense(128 * 7 * 7, activation='tanh'))
generator.add(layers.BatchNormalization())
generator.add(layers.Reshape((7, 7, 128), input_shape=(128 * 7 * 7,)))
generator.add(layers.UpSampling2D(size=(2, 2)))
generator.add(layers.Conv2D(64, (5, 5), padding='same', activation='tanh'))
generator.add(layers.UpSampling2D(size=(2, 2)))
generator.add(layers.Conv2D(1, (5, 5), padding='same', activation='tanh'))
generator.summary()


# ************************************
#      gan = generator + discriminator
# ************************************
gan = models.Sequential()
    # ***********************************
    #
    #     Fill in the blank
    #
    # ***********************************
opt_G = optimizers.RMSprop(lr=0.0004, clipvalue=1.0)
gan.compile(loss='binary_crossentropy', optimizer=opt_G)
gan.summary()

In [0]:

for epoch in range(epochs):

    for index in range(int(x_train.shape[0] / batch_size)):

        # Distriminator training on the batch
        x = x_train[index * batch_size:(index + 1) * batch_size]
        ln = x.shape[0]
        z = np.random.uniform(-1.0, 1.0, (ln, input_dim))
        gen = generator.predict(z, verbose=0)
        input_D = np.concatenate((x, gen))
        y_D = np.array([1.] * ln + [0.] * ln)
        y_D += 0.01 * np.random.random(y_D.shape)
        discriminator.trainable = True
        current_learning_rate = step_decay(epoch)
        K.set_value(discriminator.optimizer.lr, current_learning_rate)
        loss_D = discriminator.train_on_batch(input_D, y_D)

        # generator training on the batch
        z = np.random.uniform(-1.0, 1.0, (ln, input_dim))
        discriminator.trainable = False
        K.set_value(gan.optimizer.lr, current_learning_rate)
        loss_G = gan.train_on_batch(z, np.array([1] * ln))


    print('Epoch = %d, Loss D:%f,  Loss G:%f' %(epoch, loss_D, loss_G))


    if epoch % 5 == 0 or epoch == epochs - 1:
        z = np.random.uniform(-1.0, 1.0, (ln, input_dim))
        gen = gan.layers[0].predict(z, verbose=0)

        fig = plt.figure(figsize=(20, 2))
        fig.suptitle("Epoch = %d" % epoch)
        for i in range(sample_size):
            plt.subplot(1, sample_size, i + 1)
            plt.imshow(gen[i].reshape((28, 28)))


plt.show()
